In [6]:
import tensorflow as tf
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops

def zeros(shape, dtype=tf.float32, scope='default'):
    with tf.compat.v1.variable_scope(scope):
        init = tf.zeros(shape, dtype=dtype)
        return tf.Variable(init)
    
def glorot(shape, dtype=tf.float32, scope='default'):
    # Xavier Glorot & Yoshua Bengio (AISTATS 2010) initialization (Eqn 16)
    with tf.compat.v1.variable_scope(scope):
        init_range = np.sqrt(6.0 / (shape[0] + shape[1]))
        init = tf.random.uniform(
            shape, minval=-init_range, maxval=init_range, dtype=dtype)
        return tf.Variable(init)

def leaky_relu(features, alpha=0.2, name=None):
    """Compute the Leaky ReLU activation function.
    "Rectifier Nonlinearities Improve Neural Network Acoustic Models"
    AL Maas, AY Hannun, AY Ng - Proc. ICML, 2013
    http://web.stanford.edu/~awni/papers/relu_hybrid_icml2013_final.pdf
    Args:
    features: A `Tensor` representing preactivation values.
    alpha: Slope of the activation function at x < 0.
    name: A name for the operation (optional).
    Returns:
    The activation value.
    """
    with ops.name_scope(name, "LeakyRelu", [features, alpha]):
        features = ops.convert_to_tensor(features, name="features")
        alpha = ops.convert_to_tensor(alpha, name="alpha")
        return math_ops.maximum(alpha * features, features)

In [7]:
class GraphCNN_PE(object):
    def __init__(self, inputs, input_dim, hid_dims, output_dim, max_depth, act_fn, scope='gcn'):

        self.inputs = inputs

        self.input_dim = input_dim
        self.hid_dims = hid_dims
        self.output_dim = output_dim

        self.max_depth = max_depth  # hyperparam

        self.act_fn = act_fn
        self.scope = scope

        # message passing
        self.adj_mats = [tf.compat.v1.sparse_placeholder(
            tf.float32, [None, None]) for _ in range(self.max_depth)]
        self.masks = [tf.compat.v1.placeholder(
            tf.float32, [None, 1]) for _ in range(self.max_depth)]

        # initialize message passing transformation parameters
        # h: x -> x'
        self.prep_weights, self.prep_bias = \
            self.init(self.input_dim, self.hid_dims, self.output_dim)

        # f: x' -> e
        self.proc_weights, self.proc_bias = \
            self.init(self.output_dim, self.hid_dims, self.output_dim)

        # g: e -> e
        self.agg_weights, self.agg_bias = \
            self.init(self.output_dim, self.hid_dims, self.output_dim)

        # graph message passing
        self.outputs = self.forward()

    def init(self, input_dim, hid_dims, output_dim):
        # Initialize the parameters
        # these weights may need to be re-used
        # e.g., we may want to propagate information multiple times
        # but using the same way of processing the nodes
        weights = []
        bias = []

        curr_in_dim = input_dim

        # hidden layers
        for hid_dim in hid_dims:
            weights.append(
                glorot([curr_in_dim, hid_dim], scope=self.scope))
            bias.append(
                zeros([hid_dim], scope=self.scope))
            curr_in_dim = hid_dim

        # output layer
        weights.append(glorot([curr_in_dim, output_dim], scope=self.scope))
        bias.append(zeros([output_dim], scope=self.scope))

        return weights, bias

    def forward(self):
        # message passing among nodes
        # the information is flowing from leaves to roots
        x = self.inputs

        # raise x into higher dimension
        for l in range(len(self.prep_weights)):
            x = tf.matmul(x, self.prep_weights[l])
            x += self.prep_bias[l]
            x = self.act_fn(x)

        for d in range(self.max_depth):
            # work flow: index_select -> f -> masked assemble via adj_mat -> g
            y = x

            # process the features on the nodes
            for l in range(len(self.proc_weights)):
                y = tf.matmul(y, self.proc_weights[l])
                y += self.proc_bias[l]
                y = self.act_fn(y)

            # message passing
            y = tf.sparse.sparse_dense_matmul(self.adj_mats[d], y)

            # aggregate child features
            for l in range(len(self.agg_weights)):
                y = tf.matmul(y, self.agg_weights[l])
                y += self.agg_bias[l]
                y = self.act_fn(y)

            # remove the artifact from the bias term in g
            y = y * self.masks[d]

            # assemble neighboring information
            x = x + y

        return x


In [14]:
# pe predict
pe_inputs = np.zeros((5, 4), dtype=np.float32)
pe_input_dim = 4
hid_dims = [16,8]
output_dim = 8
max_depth = 2
act_fn = leaky_relu
scope='gcn_pe'


In [15]:
gcn_pe = GraphCNN_PE(pe_inputs, pe_input_dim, hid_dims, output_dim, max_depth, act_fn, scope)

In [18]:
sess = tf.Session()
gcn_pe

In [19]:
# pe gradient
pe_inputs = np.zeros((16, 4), dtype=np.float32)
pe_input_dim = 4
hid_dims = [16,8]
output_dim = 8
max_depth = 2
act_fn = leaky_relu
scope='gcn_pe'


In [20]:
gcn_pe = GraphCNN_PE(pe_inputs, pe_input_dim, hid_dims, output_dim, max_depth, act_fn, scope)